In [ ]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from prediction_utils.extraction_utils.database import BQDatabase

In [ ]:
config_dict = {
    'gcloud_project': 'som-nero-phi-nigam-optum',
    'dataset_project': 'som-rit-starr-prod',
    'dataset': 'optum_dod_2019q1_cdm_53',
    'cohort_project': 'som-nero-phi-nigam-optum',
    'cohort_dataset': 'ascvd_cohort_tables',
    'cohort_name': 'ascvd_10yr_optum_dod_2019q1_cdm_53_sampled'
}

In [ ]:
db = BQDatabase(gcloud_project=config_dict['gcloud_project'])

In [ ]:
query = """
    SELECT  * 
    FROM `ascvd_cohort_tables.ascvd_10yr_optum_dod_2019q1_cdm_53_sampled_pcevar` 
    WHERE has_statin_history=0 and has_cvd_history=0
"""
extracted_cohort = db.read_sql_query(query)

In [ ]:
extracted_cohort.query('ldl_value_as_number == 0').shape

In [ ]:
assert (extracted_cohort.ldl_measurement_date > extracted_cohort.index_date).sum() == 0

### Clean up the LDL values
    * Consider LDL == 0 as invalid
    * Consider LDL < 10 or > 500 mg/dL to be invalid

In [ ]:
extracted_cohort.loc[
    extracted_cohort['ldl_value_as_number'] == 0,
    'ldl_value_as_number'
] = np.nan

In [ ]:
extracted_cohort.loc[
    (extracted_cohort['ldl_value_as_number'] < 10) | (extracted_cohort['ldl_value_as_number'] > 500), 
    'ldl_value_as_number'
] = np.nan

In [ ]:
cohort_ldl = extracted_cohort.query('~ldl_value_as_number.isna()')

In [ ]:
# Note that all LDL units are in mg/dL
print(cohort_ldl.groupby('ldl_unit_concept_id').agg('size'))

In [ ]:
cohort_ldl.groupby('ldl_concept_id').agg('size')

In [ ]:
columns = ['prediction_id', 'person_id', 'index_date', 'ldl_value_as_number', 'ldl_unit_concept_id', 'ldl_concept_id']
cohort_ldl.query('ldl_value_as_number > 0').sort_values('ldl_value_as_number')[columns].head(100)

In [ ]:
# Convert LDL values to mmol/L
cohort_ldl = (
    cohort_ldl
    .assign(ldl_value_as_number=lambda x: x.ldl_value_as_number/38.67)
)

In [ ]:
# Plot the LDL values
plt.hist(cohort_ldl.ldl_value_as_number.values)

In [ ]:
cohort_ldl = cohort_ldl[['prediction_id', 'ldl_value_as_number']]

#### Read in the set of predictions for the selected ERM models

In [ ]:
selected_configs_path = '/local-scratch/nigam/secure/optum/spfohl/zipcode_cvd/optum/dod/experiments/ascvd_10yr_optum_dod_selected/selected_configs.csv'


In [ ]:
selected_config_df = pd.read_csv(selected_configs_path)

In [ ]:
selected_config_filenames = selected_config_df.query('tag == "erm_baseline"').config_filename.unique()

In [ ]:
output_df_path = '/local-scratch/nigam/secure/optum/spfohl/zipcode_cvd/optum/dod/experiments/ascvd_10yr_optum_dod_erm_tuning/performance/'

In [ ]:
output_df_dict = {
    config_filename: pd.read_parquet(os.path.join(output_df_path, config_filename, 'output_df.parquet'))
    for config_filename in selected_config_filenames
}

In [ ]:
output_df = pd.concat(output_df_dict)

In [ ]:
output_df = output_df.reset_index(level=-1, drop=True).rename_axis('config_filename').reset_index()

In [ ]:
output_df

In [ ]:
# Take the mean prediction over training replicates
result = output_df.query('phase == "test"').groupby(['row_id'])[['weights', 'pred_probs']].agg('mean').reset_index()

In [ ]:
# Read the cohort metadata in
cohort_path = '/local-scratch/nigam/secure/optum/spfohl/zipcode_cvd/optum/dod/cohort/cohort_fold_1_5_ipcw.parquet'
cohort_df = pd.read_parquet(cohort_path)

In [ ]:
cohort_df.query('~ascvd_binary.isnull() & phase == "test"')

In [ ]:
cohort_df = cohort_df.merge(result).merge(cohort_ldl)

In [ ]:
cohort_df

In [ ]:
# Fit an unweighted LR model
x = np.log(cohort_df.pred_probs.values)
x = sm.add_constant(x)
y = cohort_df.ldl_value_as_number.values
model = sm.OLS(y, x)
model_result=model.fit()
model_result.summary()

In [ ]:
# Fit a weighted LR model
x = np.log(cohort_df.pred_probs.values)
x = sm.add_constant(x)
y = cohort_df.ldl_value_as_number.values
model = sm.WLS(y, x, weights = cohort_df.weights.values)
model_result=model.fit()
model_result.summary()

In [ ]:
reg_x = np.linspace(0.001, 0.4)
reg_y = np.log(reg_x)*model_result.params[-1] + model_result.params[0]

In [ ]:
plt.scatter(cohort_df.pred_probs, cohort_df.ldl_value_as_number)
plt.plot(reg_x, reg_y, lw=2, color='r')
plt.xlabel('Predicted 10-year ASCVD risk')
plt.ylabel('LDL (mmol/L)')

In [ ]:
## Assume a constant model
ldl_mean = np.average(cohort_df.ldl_value_as_number, weights=cohort_df.ipcw_weight)

In [ ]:
ldl_mean_group = cohort_df.groupby(['race_eth']).apply(lambda x: np.average(x.ldl_value_as_number, weights=x.ipcw_weight))
ldl_mean_group

In [ ]:
figure_path = '../zipcode_cvd/experiments/figures/optum/'

In [ ]:
plt.scatter(cohort_df.pred_probs, cohort_df.ldl_value_as_number, s=5, color='#4daf4a')
plt.axhline(ldl_mean, lw=3, color='#e41a1c', label='Constant')
plt.plot(reg_x, reg_y, lw=3, color='#377eb8', label=r'OLS ($R^2=0.004$)')
plt.xlabel('Predicted 10-year ASCVD risk', fontsize=14)
plt.ylabel('LDL-C (mmol/L)', fontsize=14)
plt.legend()
sns.despine()
# plt.savefig(os.path.join(figure_path, 'ldl_scatter.png'), dpi=180, bbox_inches='tight')
# plt.savefig(os.path.join(figure_path, 'ldl_scatter.pdf'), bbox_inches='tight')

### Model for risk reduction, following Soran et al
    * Soran, H., Schofield, J. D., & Durrington, P. N. (2015). Cholesterol, not just cardiovascular risk, is important in deciding who should receive statin treatment. European Heart Journal, 36(43), 2975–2983. https://doi.org/10.1093/EURHEARTJ/EHV340

In [ ]:
ldl_rr = 0.43
ldl_abs_reduction = ldl_rr*ldl_mean
cvd_rr = (1-0.78**(ldl_rr*ldl_mean))
print(f'Relative risk reduction: {cvd_rr}')

In [ ]:
ldl_rr = 0.43
ldl_abs_reduction = ldl_rr*ldl_mean_group
cvd_rr = (1-0.78**(ldl_rr*ldl_mean_group))
cvd_rr
# print(f'Relative risk reduction: {cvd_rr}')